In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from peft import PeftModel, PeftConfig

# 1. 指向你截图中的那个文件夹路径
lora_model_path = "./distilbert-base-uncased-finetuned-LoRA-WNLI"

# 2. 自动从 adapter_config.json 中读取底座模型的名字
# 这一步会发现底座模型是 "distilbert-base-uncased"
config = PeftConfig.from_pretrained(lora_model_path)


In [12]:
config.base_model_name_or_path


'./distilbert-base-uncased'

In [10]:
import os

In [15]:
# 3. 加载底座模型 (Base Model)
# 注意：如果是分类任务，Num_labels 要与你微调时一致
base_model = AutoModelForSequenceClassification.from_pretrained(config.base_model_name_or_path)
# 4. 加载 LoRA 权重 (Adapter)
model = PeftModel.from_pretrained(base_model, lora_model_path)


loading configuration file ./distilbert-base-uncased/config.json
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.57.1",
  "vocab_size": 30522
}

loading weights file ./distilbert-base-uncased/model.safetensors
Some weights of the model checkpoint at ./distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint 

In [16]:
# 5. 加载分词器 (Tokenizer)
# 截图显示文件夹里有 tokenizer.json 等文件，直接从该目录加载即可
tokenizer = AutoTokenizer.from_pretrained(lora_model_path)

# 此时的 model 已经加载了 LoRA 权重，可以进行推理了
model.eval()

loading file vocab.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading file chat_template.jinja


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): DistilBertForSequenceClassification(
      (distilbert): DistilBertModel(
        (embeddings): Embeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (transformer): Transformer(
          (layer): ModuleList(
            (0-5): 6 x TransformerBlock(
              (attention): DistilBertSdpaAttention(
                (dropout): Dropout(p=0.1, inplace=False)
                (q_lin): lora.Linear(
                  (base_layer): Linear(in_features=768, out_features=768, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=7

In [17]:
from datasets import load_dataset

# 下载并加载 GLUE 的 WNLI 子集
dataset = load_dataset("glue", "wnli", cache_dir='./dataset')

# 查看数据集结构
print(dataset)

# 打印一个训练集样本
print(dataset["train"][0])

README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/38.8k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/11.1k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/13.6k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/635 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/71 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/146 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 635
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 71
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 146
    })
})
{'sentence1': 'I stuck a pin through a carrot. When I pulled the pin out, it had a hole.', 'sentence2': 'The carrot had a hole.', 'label': 1, 'idx': 0}


In [18]:
import torch
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from peft import PeftModel
from torch.utils.data import DataLoader
from tqdm import tqdm

test_data = dataset["validation"]  # 如果要生成提交文件，请换成 dataset["test"]

# 4. 预处理函数
def preprocess_function(examples):
    return tokenizer(examples["sentence1"], examples["sentence2"], 
                     truncation=True, padding="max_length", max_length=128)

tokenized_test = test_data.map(preprocess_function, batched=True)
tokenized_test.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# 5. 推理循环
predictions = []
references = []

dataloader = DataLoader(tokenized_test, batch_size=16)

print("正在进行推理...")
for batch in tqdm(dataloader):
    inputs = {k: v.to(model.device) for k, v in batch.items() if k != "label"}
    with torch.no_grad():
        outputs = model(**inputs)
    
    logits = outputs.logits
    preds = torch.argmax(logits, dim=-1)
    predictions.extend(preds.cpu().numpy())
    references.extend(batch["label"].cpu().numpy())

# 6. 计算准确率 (仅适用于 validation)
correct = sum(1 for p, r in zip(predictions, references) if p == r)
accuracy = correct / len(references)
print(f"\nValidation Accuracy: {accuracy:.4f}")

Map:   0%|          | 0/71 [00:00<?, ? examples/s]

正在进行推理...


100%|██████████| 5/5 [00:01<00:00,  2.96it/s]


Validation Accuracy: 0.5211
